In [1]:
import boto3
import sagemaker
import io

import pandas as pd
import numpy as np

s3_client = boto3.client('s3')
s3_bucket_name = 'awsml-sagemaker-test-source629'

response = s3_client.get_object(Bucket='awsml-sagemaker-test-source629', Key='iris_train.csv')
response_body = response["Body"].read()
df_iris_train = pd.read_csv(io.BytesIO(response_body), header=0, delimiter=",", low_memory=False)

response = s3_client.get_object(Bucket='awsml-sagemaker-test-source629', Key='iris_test.csv')
response_body = response["Body"].read()
df_iris_test = pd.read_csv(io.BytesIO(response_body), header=0, index_col=False, delimiter=",", low_memory=False)

df_iris_features_train = df_iris_train.iloc[:,1:]
df_iris_target_train = df_iris_train.iloc[:,0]

df_iris_features_test = df_iris_test.iloc[:,1:]
df_iris_target_test = df_iris_test.iloc[:,0]

from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=3)
kmeans_model.fit(df_iris_features_train)

kmeans_predictions = kmeans_model.predict(df_iris_features_test)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [2]:
print(kmeans_predictions)

[1 2 0 2 2 2 2 0 2 2 0 1 2 1 0 1 1 0 0 0 2 1 2 2 2 2 2 2 1 2 1 2 1 1 2 2 2
 0]


In [3]:
print(df_iris_target_test.values.ravel())

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-virginica' 'Iris-virginica' 'Iris-virginica'
 'Iris-virginica' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-virginica']


In [4]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
labelEncoder.fit(df_iris_target_test)
df_iris_target_test = labelEncoder.transform(df_iris_target_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm_kmeans = confusion_matrix(df_iris_target_test, kmeans_predictions)
acc_kmeans = accuracy_score(df_iris_target_test, kmeans_predictions)

In [6]:
print(cm_kmeans)
print(acc_kmeans)

[[ 0 10  0]
 [ 0  0 15]
 [ 8  0  5]]
0.13157894736842105
